In [2]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch

base_model_name = "unsloth/Llama-3.2-3B-Instruct-unsloth-bnb-4bit"
local_model_name = "3B-GRPO/3B-GRPO-checkpoint-1000"
model_name = local_model_name

tokenizer = AutoTokenizer.from_pretrained(local_model_name)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=local_model_name,
    max_seq_length=2500,
    load_in_4bit=True,
    fast_inference=True,
    gpu_memory_utilization=0.8,
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.413 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading 3B-JOB-ckpts/ckpt-4296 with actual GPU utilization = 79.49%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 47.41 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2500. Num Sequences = 320.
Unsloth: vLLM's KV Cache can use up to 31.52 GB. Also swap space = 6 GB.
INFO 04-05 20:22:35 config.py:549] This model supports multiple tasks: {'embed', 'classify', 'score', 'generate', 'reward'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load

[W405 20:22:36.316202794 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.37it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.37it/s]



INFO 04-05 20:22:37 model_runner.py:1115] Loading model weights took 2.1360 GB
INFO 04-05 20:22:37 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-05 20:22:38 worker.py:267] Memory profiling takes 0.84 seconds
INFO 04-05 20:22:38 worker.py:267] the current vLLM instance can use total_gpu_memory (47.41GiB) x gpu_memory_utilization (0.79) = 37.69GiB
INFO 04-05 20:22:38 worker.py:267] model weights take 2.14GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.49GiB; the rest of the memory reserved for KV Cache is 34.00GiB.
INFO 04-05 20:22:38 executor_base.py:111] # cuda blocks: 19896, # CPU blocks: 3510
INFO 04-05 20:22:38 executor_base.py:116] Maximum concurrency for 2500 tokens per request: 127.33x
INFO 04-05 20:22:40 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error

Capturing CUDA graph shapes: 100%|██████████| 43/43 [00:25<00:00,  1.71it/s]

INFO 04-05 20:23:05 model_runner.py:1562] Graph capturing finished in 25 secs, took 0.76 GiB
INFO 04-05 20:23:05 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 28.00 seconds



Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128009)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Li

In [ ]:

# ✅ 3. 推理函数

gbnf_grammar = """
<sql_hint> ::= "/*+ " <hint_list> " */"
<hint_list> ::= <hint> (" " <hint>)*

<hint> ::= <join_hint> | <scan_hint> | <leading_hint>

<join_hint> ::= "MergeJoin(" <table_list> ")" 
              | "NestLoop(" <table_list> ")" 
              | "HashJoin(" <table_list> ")"

<scan_hint> ::= "SeqScan(" <table_name> ")" 
              | "IndexScan(" <table_name> ")"

<leading_hint> ::= "Leading(" <nested_tables> ")"

<table_list> ::= <table_name> (" " <table_name>)*
<table_name> ::= [a-zA-Z0-9_]+
<nested_tables> ::= "(" <table_or_group> ")"
<table_or_group> ::= <table_name> 
                   | "(" <table_or_group> " " <table_or_group> ")"
"""

def run_inference(prompt: str, max_new_tokens: int = 64):
    """ 使用微调后的模型进行推理，返回生成的文本 """
    messages = [
        {"role": "user", "content": prompt}
    ]

    # ✅ 确保 tokenizer 返回字典格式
    inputs = tokenizer(
        tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        ),
        return_tensors="pt"
    ).to("cuda" if torch.cuda.is_available() else "cpu")

    # ✅ 4. 运行推理
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        max_new_tokens=max_new_tokens,
        temperature=0.7,          # 控制生成的随机性"
        top_p=0.9,               # nucleus sampling
        repetition_penalty=1.1,  # 减少重复生成
        use_cache=True,
    )

    # ✅ 5. 解码输出
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# ✅ 6. 测试推理
test_prompt = "解释深度学习中的卷积神经网络，并给出一个示例。"
result = run_inference(test_prompt, max_new_tokens=1280)
print(f"\n=== 🚀 Inference Output ===\n{result}")

In [ ]:
test_prompt = """
# You are a Database Assistant which help the internal decide the join order. Based on the database schema are some cardinality of the filters, you will construct a query plan tree for the internal to execute. 
Please Respond in the following format:
<think>...</think>and directly give the final answer within the example format <answer>...</answer>

# You can do these actions:
Check whether there are any remaining subtrees and tables haven't been joined.
Decide whether to divide the candidate table set into two subset, and tackle each of them recursively(this is because the action space is a bushy tree)
Pick one table from the candidate set(or subset), assign an "IndexScan", "IndexOnlyScan" or "SeqScan" label for it.
Join one table(or an existing subtree, with a label) to an existing subtree, or another table, and assign a "HashJoin", "NestLoop" or "MergeJoin" label for that node.

# You have these restrictions on your actions:
Each table (may with a unique alias or just the table name) should be picked and picked only once.
The table must be picked before Join.
Each subtree/table could be joined only once, and after the join operation a new subtree appears, the old two are combined.
Count on the newest subtree appear in each join step in the check operation.
After join operation, you must do the check operation, to check if there exists any subtrees and tables haven't been joined.
After check operation, you must do the decide operation, to tackle the untouched tables and used tables separately. Unless all the tables and subtrees have been tackled.

# Here is one example:
## You have these tables:
tag AS t1
site AS s1
question AS q1
tag_question AS tq1
so_user AS u1
tag AS t2
site AS s2
question AS q2
tag_question AS tq2
so_user AS u2
account AS account

## table cardinality = {
    'tag': 186770,
    'site': 173,
    'question': 12663270,
    'tag_question': 36885060,
    'so_user': 21096884,
    'account': 13863416,
}

## filter cardinality = {
    ('site AS s1', '(s1.site_name='stats')'): 1,
    ('tag AS t1', '(t1.name  = 'machine-learning')'): 3,
    ('site AS s2', '(s2.site_name='stackoverflow')'): 1,
    ('tag AS t2', '(t2.name  = 'heroku')'): 3,
}

Analyzes the given tables and filter conditions to determine the best join order.
<answer>/*+ NestedLoop(t1 s1 tq1 q1 u1 t2 s2 tq2 q2 u2 account)
 HashJoin(t1 s1 tq1 q1 u1 t2 s2 tq2 q2 u2)
 NestedLoop(t2 s2 tq2 q2 u2)
 NestedLoop(t2 s2 tq2 q2)
 NestedLoop(t2 s2 tq2)
 MergeJoin(t2 s2)
 NestedLoop(t1 s1 tq1 q1 u1)
 NestedLoop(t1 s1 tq1 q1)
 NestedLoop(t1 s1 tq1)
 MergeJoin(t1 s1)
 SeqScan(t1)
 SeqScan(s1)
 IndexOnlyScan(tq1)
 IndexScan(q1)
 IndexScan(u1)
 SeqScan(t2)
 SeqScan(s2)
 IndexOnlyScan(tq2)
 IndexScan(q2)
 IndexScan(u2)
 IndexScan(account)
 Leading(((((((t1 s1) tq1) q1) u1) ((((t2 s2) tq2) q2) u2)) account)) */</answer>

## Here is the question you need to solve. Please provide the hint as the example shows.
# You have these tables:
title AS t
kind_type AS kt
info_type AS it1
movie_info AS mi1
movie_info AS mi2
info_type AS it2
cast_info AS ci
role_type AS rt
name AS n
movie_keyword AS mk
keyword AS k

# table cardinality = {
    'title': 2528553,
    'kind_type': 7,
    'info_type': 113,
    'movie_info': 14814177,
    'cast_info': 36244344,
    'role_type': 12,
    'name': 4167491,
    'movie_keyword': 4523930,
    'keyword': 134170,
}

# filter cardinality = {
    ('info_type AS it1', '((it1.id in ('7')))'): 1,
    ('info_type AS it2', '((it2.id in ('8')))'): 1,
    ('movie_info AS mi1', '((mi1.info in ('MET:600 m','OFM:35 mm','PCS:Spherical','PFM:35 mm','RAT:1.37 : 1')))'): 706025,
    ('movie_info AS mi2', '((mi2.info in ('France','Germany','Japan','Mexico','Portugal','Spain','UK','USA')))'): 929343,
    ('kind_type AS kt', '((kt.kind in ('tv series','video game','video movie')))'): 3,
    ('role_type AS rt', '((rt.role in ('producer')))'): 1,
    ('name AS n', '((n.gender in ('m') OR n.gender IS NULL))'): 2591471,
    ('title AS t', '((t.production_year <= 2000)) AND ((t.production_year >= 1925))'): 970437,
}
"""
result = run_inference(test_prompt, max_new_tokens=5000)
print(f"\n=== 🚀 Inference Output ===\n{result}")

In [ ]:
gbnf_grammar = """
<sql_hint> ::= "/*+ " <hint_list> " */"
<hint_list> ::= <hint> (" " <hint>)*
<hint> ::= <join_hint> | <scan_hint> | <leading_hint>
<join_hint> ::= "MergeJoin(" <table_list> ")" 
              | "NestLoop(" <table_list> ")" 
              | "HashJoin(" <table_list> ")"
<scan_hint> ::= "SeqScan(" <table_name> ")" 
              | "IndexScan(" <table_name> ")"
<leading_hint> ::= "Leading(" <nested_tables> ")"
<table_list> ::= <table_name> (" " <table_name>)*
<table_name> ::= [a-zA-Z0-9_]+
<nested_tables> ::= "(" <table_or_group> ")"
<table_or_group> ::= <table_name> 
                   | "(" <table_or_group> " " <table_or_group> ")"
"""

# 构造 prompt，要求输出必须严格按照 JSON 格式生成：
# 字段 "grammar" 对应上面的语法文本，
# 字段 "example" 是一个符合该语法的 SQL Hint 示例
prompt = f"""请根据下面的 GBNF SQL Hint 语法生成输出。
要求输出为 JSON 格式，并且必须包含两个字段：
  1. "grammar"：值为下面提供的语法；
  2. "example"：值为一个符合该语法的 SQL Hint 示例，例如：/*+ SeqScan(table1) */
字段顺序必须固定为 "grammar" 在前，"example" 在后，且输出中不能包含其他内容。

GBNF 语法:
{gbnf_grammar}
"""
result = run_inference(prompt, max_new_tokens=5000)
print(f"\n=== 🚀 Inference Output ===\n{result}")